#### Auteurs:
- *DIAGNE Elhadji Magatte*
- *DJIMENE TSOTSOP Staelle*
- *DONGMO TIOGUIM Gilchrist Loïc*

#### Nettoyage et fusion des bases obtenues apres webscrapping

In [ ]:
df=pd.read_csv('https://raw.githubusercontent.com/Gil97-tech/TP2A/master/final_dataListing.csv')

In [ ]:
df2=pd.read_csv('https://raw.githubusercontent.com/Gil97-tech/TP2A/master/final_database.csv')

In [ ]:
## Colonne du nombre de commentaires, on retire les parentheses et on transforme en numeque

df2['nbComments'] = df2['nbComments'].str.replace(r'\D','')

df2['nbComments']=np.where(df2['nbComments']=='',0,df2['nbComments'])

df2.loc[:, "nbComments"] = pd.to_numeric(df2.nbComments)

df2.sort_values(['nbComments'], ascending=False)

In [ ]:
#Colonne des evaluations

df2['evaluation']=np.where(df2['evaluation']=="pas d'évaluation",0,df2['evaluation'])

df2.loc[:, "evaluation"] = pd.to_numeric(df2.evaluation)


In [ ]:
## Traitement de la colonne des prix:

df2['prix']=df2['prix'].str.replace(":", "/").str.split("/",n = 0, expand = True)[1]
df2['prix']=df2['prix'].str.replace('€','')
df2['prix']=df2['prix'].str.replace('Prix réduit','')
df2['prix']=df2['prix'].str.replace('Discounted price','')

## Conversion en une monnaie unique notamment en Euros

def conversion(df):
    n=len(df)
    for i in range(n):
        try:
            df[i]=float(df[i])
        except:
            df[i]=1.21*float(df[i].strip('$'))
    return df
df2['prix']=conversion(df2['prix'])


In [ ]:
## Identifiant des listings dans la base
n=range(1,286)
df2.drop(['Unnamed: 0'], inplace=True, axis=1)

df2['numero_listing']=n

##Selection des listings avec les  commentaires

df_wcomment=df2[df2['nbComments']!=0].sort_values(['nbComments'], ascending=False)


In [ ]:
# jointures des deux bases pour avoir une base des listings avec les commentaires pour chaque listing
df3 = pd.merge(df2[['title', 'nbComments', 'evaluation',
                          'topRow', 'prix','numero_listing']], df,
                left_on='numero_listing',right_on ='index', how='left')

In [ ]:
#Traitement de la colonne des caracteristiques des logements

def cleanRoomInfo(df):
    df.loc[:, "guests"] = df.loc[:, "roomInfo"].str.split(" · ", n = 0, expand = True)[0].str.replace(" guest", "").str.replace("s", "")
    df.loc[:, "bedrooms"] = df.loc[:, "roomInfo"].str.split(" . ", n = 0, expand = True)[1].str.replace(" bedroom", "").str.replace("s", "")
    df.loc[:, "beds"] = df.loc[:, "roomInfo"].str.split(" . ", n = 0, expand = True)[2].str.replace(" bed", "").str.replace("s", "")
    df.loc[:, "bathrooms"] = df.loc[:, "roomInfo"].str.split(" . ", n = 0, expand = True)[3].str.replace(" bath", "").str.replace("s", "")
    df.loc[:, "guests"] = df.loc[:, "guests"].str.replace(" voyageur", "").str.replace("s", "") 
    df.loc[:, "bedrooms"] = df.loc[:, "bedrooms"].str.replace(" chambre", "").str.replace("s", "") 
    df.loc[:, "beds"]=df.loc[:, "beds"].str.replace(" lit", "").str.replace("s", "") 
    df.loc[:, "bathrooms"]=df.loc[:, "bathrooms"].str.replace(" alle de bain", "").str.replace("s", "")
    df.loc[:, "guests"] = pd.to_numeric(df.guests, errors = 'coerce')
    df.loc[:, "beds"] = pd.to_numeric(df.beds, errors = 'coerce')
    df.loc[:, "bedrooms"] = pd.to_numeric(df.bedrooms.str.split(" ", n = 0, expand = True)[0], errors = "coerce")
    df.loc[:, "bathrooms"] = pd.to_numeric(df.bathrooms.str.split(" ", n = 0, expand = True)[0], errors = "coerce")
    return df.drop("roomInfo", axis = 1)

